In [ ]:
# define
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd
import re
import json
import os 
import time
import datetime
import numpy as np
from selenium import webdriver 
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import Chrome
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains

import jieba
import jieba.analyse
from pymongo import MongoClient
mongodb_atlas_account = "adan7575"
mongodb_atlas_password = "adan7575"
ss = requests.session()


In [ ]:
def get_typhoon_alert():
    print("typhoon data crawler -> start")
    header_typhoon = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36",
    "Cookie": "PHPSESSID=ck51k725hgodfmfd7cbi4b0ks4; _gid=GA1.3.1183769150.1622787669; _ga=GA1.3.25450972.1620709885; _ga_K6HENP0XVS=GS1.1.1622787669.3.1.1622787695.0; TS01b0fe7f=0107dddfefcf72bbe6298d9f6067078ff9f4c14164221cb96410f497cf4481230f20f5073ca7ae71a4a5fe265de60a5c20c91db504",
    "X-Requested-With": "XMLHttpRequest"}
    url_typhoon = "https://rdc28.cwb.gov.tw/TDB/public/warning_typhoon_list/get_warning_typhoon"
    cols_typhoon = ['颱風編號',
         '中文名稱',
         '英文名稱',
         '侵臺路徑分類',
         '海上警報開始時間',
         '近臺強度',
         '近臺最低氣壓(hPa)',
         '近臺最大風速(m/s)',
         '近臺7級風暴風半徑(km)',
         '近臺10級風暴風半徑(km)',
         '海上警報結束時間',       
         '警報發布報數']
    
    res = ss.post(url_typhoon, headers=header_typhoon)
    data = res.text[1:]
    json_data = json.loads(data)
    df = pd.json_normalize(json_data)
    # 部分字串型態欄位轉換成數字型態
    int_col = ['id', 'official_path_category', 'min_pressure', 'max_wind_speed', 'max_range7', 'max_range10', 'warning_count']
    df[int_col] = df[int_col].apply(pd.to_numeric)
    print("typhoon data crawler -> finish")
    
    # 存入mongodb Database->TWFruits, collections->typhoon
    client = MongoClient('mongodb+srv://{}:{}@twfruit.i2omj.mongodb.net/myFirstDatabase?retryWrites=true&w=majority'.format(mongodb_atlas_account, mongodb_atlas_password))
    db = client.TWFruits
    typhoon = db.climate_typhoon_alert
    
    print("typhoon data update to mongodb -> start")
    for excist_id in df['id']:
        if [x for x in typhoon.find({'id':excist_id})] == []:
            typhoon_update = df.loc[df["id"]==excist_id].to_dict(orient='records')
            updated = typhoon.insert_one(typhoon_update[0]).inserted_id
#             print(typhoon_update)
            print("typhoon data update id ", updated)
    
    print("typhoon data update to mongodb -> finish")
    client.close() 
    
get_typhoon_alert()

In [ ]:
def produce_year_data():
    
    print('produce_year_data crawler start')  
    url = "https://data.coa.gov.tw/Service/OpenData/DataFileService.aspx?UnitId=135" 
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36'
            }

    res = requests.get(url, headers=headers)
    data = json.loads(res.text)
    df = pd.json_normalize(data) # normalize => 將json檔案攤平，如有巢狀結構的話
    print('produce_year_data crawler finish')  
    
    wanted_col = ['年度', '地區別', '果品類別', '收穫株數', '收穫面積_公頃', '產量_公噸']
    df = df[wanted_col]
    df = df.loc[~(df["地區別"].isin(["臺灣省","福建省"]))]

    # 去除含有字串的資料
    target_col = ['收穫株數', '收穫面積_公頃', '產量_公噸']
    df.loc[(df["收穫面積_公頃"].str.contains(r"[A-Z-]", na=False)), target_col] = 0
    df[target_col] = df[target_col].apply(pd.to_numeric)

    client = MongoClient('mongodb+srv://{}:{}@twfruit.i2omj.mongodb.net/myFirstDatabase?retryWrites=true&w=majority'.format(mongodb_atlas_account, mongodb_atlas_password))
    db = client.TWFruits
    fruit_produce_year = db.produce_yearly_fruit_data

    print("produce_year data update to mongodb -> start")
    fruit_produce_year_update = df.to_dict(orient='records')
    updated = fruit_produce_year.insert_many(fruit_produce_year_update)

    print("produce_year data update to mongodb -> finish")
    client.close() 
    return

produce_year_data()

In [ ]:
def title_mining(tmp_t):
    return jieba.analyse.extract_tags(tmp_t, topK=4, withWeight=True, allowPOS=('n', 'nr', 'ns', 'nz', 'v', 'vd', 'vn'))

def content_mining(tmp_c):
    return jieba.analyse.extract_tags(tmp_c, topK=20, withWeight=True, allowPOS=('n', 'nr', 'ns', 'nz', 'v', 'vd', 'vn'))

In [ ]:
def news_jieba(function_name, ID_jieba, title_jieba, content_jieba):
    content_list = []
    title_list = []
    
    # 因為有不同文章來源，為了區別使用爬蟲網站縮寫+_id, coa=農委會, afa=農糧署
    title_col = [function_name+'_id','key_1', 'value_1', 'key_2', 'value_2', 'key_3', 'value_3', 'key_4', 'value_4']
    content_col = [function_name+'_id','key_1', 'value_1', 'key_2', 'value_2', 'key_3', 'value_3', 'key_4', 'value_4', 'key_5', 'value_5',
            'key_6', 'value_6', 'key_7', 'value_7', 'key_8', 'value_8', 'key_9', 'value_9', 'key_10', 'value_10',
            'key_11', 'value_11', 'key_12', 'value_12', 'key_13', 'value_13', 'key_14', 'value_14', 'key_15', 'value_15',
            'key_16', 'value_16', 'key_17', 'value_17', 'key_18', 'value_18', 'key_19', 'value_19', 'key_20', 'value_20']

    for i in range(len(ID_jieba)):
        # title與content分別進入Text mining function處理
        tmp_title = title_mining(title_jieba[i])
        tmp_content = content_mining(content_jieba[i])
        
        # 清空list
        title_keyword = []
        content_keyword = []
        
        # 第一欄加入文章ID以辨識此欄位是哪篇文章
        title_keyword.append(int(ID_jieba[i]))
        content_keyword.append(int(ID_jieba[i]))
        
        # jieba分詞完是一個tuple包含分詞與詞頻的狀態，為了方便存取，將兩者拆開
        for i in range(4):
            # 標題
            if i >= len(tmp_title):
                # 若標題找到的關鍵字小於4個(topK=4)則key填入"NA",value填0,若這邊是空值會導致dataframe columns長度不符
                title_keyword.append('NA')
                title_keyword.append(0)    
            else:
                title_keyword.append(tmp_title[i][0]) # 分詞 str
                title_keyword.append(tmp_title[i][1]) # 詞頻 float
                
        # 若內文找到的關鍵字小於20個(topK=20)則則key填入"NA",value填0,若這邊是空值會導致dataframe columns長度不符
        for i in range(20):
            # 內容
            if i >= len(tmp_content):
                title_keyword.append('NA')
                title_keyword.append(0)
            else:
                content_keyword.append(tmp_content[i][0]) # 分詞 str
                content_keyword.append(tmp_content[i][1]) # 詞頻 float
        #此篇文章處理完就加入list中 -> [[文章1],[文章2]...]
        title_list += [title_keyword]   
        content_list += [content_keyword]
        
    # 建立DataFrame並return
    df_title = pd.DataFrame(np.array(title_list), columns=title_col)
    df_content = pd.DataFrame(np.array(content_list), columns=content_col)
   
    return(df_title, df_content)


In [ ]:
def afa_news(page_tmp):
    
    print('afa_news crawler start')  
    def get_info(sub_soup):
        # 進入function後每一個try都會執行，若發生錯誤或取不到值則讓他顯示 Error方便debug
        # 對照圖(二)單則農業新聞
        try:               
            title_t = sub_soup.select('div[class="col-sm-9"]')[0].text
        except:
            title_t = 'title Error'
        try:                
            content_t = sub_soup.select('article[class="shared-content-text"]')[0].text
        except:
            content_t = 'content Error'
        try:              # text會取到"發布日期：110-06-02"，後面再用split切割成 ["發布日期：", "110-06-02"]，取第二個
            post_date = sub_soup.select('div[class="agricultural-news-content-title row mb-lg"]')[0].text.split('發布日期：')[1]
        except:
            post_date = 'post_date Error'

        # return只會傳str，需要將上面取得之內容放進list內整個回傳， 否則會只回傳第一個字元     
        tmp_list = [re.sub('[-:_、【】。；：)(「」，.&+\n\t\r\u3000]', ' ',title_t),
                    re.sub('[-:_、【】。；：)(「」，.&+\n\t\r\u3000]', ' ', content_t),
                    re.sub('-','/',post_date)]
        return tmp_list[0], tmp_list[1], tmp_list[2]

    url = []
    link = []
    out = []
    title = []
    content = []
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36',
           }
    org_url = 'https://www.afa.gov.tw/cht/index.php?code=list&ids=307'
#     page_tmp = 1
    for i in range(1, page_tmp+1):
        url.append(org_url+'&page={}'.format(i))

    # 從主頁面get request並用BeautifulSoup轉換成html，用開發工具發現子頁面的連結在a標籤的'article_class'屬性中
    # 存取該標籤的'href'，並存到link中
    for page in range(len(url)):
        print('page=', page+1)
        res = ss.get(url=url[page], headers=headers)
        soup = BeautifulSoup(res.text, 'html.parser')

        # 對照圖(一)農業新聞主頁面的開發者工具
        # 把要爬的所有子頁面的連結都先存起來
        for i in range(0, len(soup.select('a[class="article_class"]'))):
            link.append(soup.select('a[class="article_class"]')[i]['href'])
            res_sub = ss.get(url=link[i], headers=headers)
            soup_sub = BeautifulSoup(res_sub.text, 'html.parser')        
#     print('link=', link)

    #將link中的article_id當成存進資料庫後的唯一識別
    ID = list(map(lambda x: x.split('&article_id=')[1], link))    
    print('ID=', ID)

    # 透過個連結逐一訪問子頁面
    for j in range(len(link)):
        print('進入子新聞頁面', j+1)
        res_sub = ss.get(url=link[j], headers=headers)
        sub_soup_main = BeautifulSoup(res_sub.text, 'html.parser')

        # 將BeautifulSoup處理過的html代入函式處理，主要程式流程看起來比較乾淨
        # html帶入get_info執行完會回傳3個str，分別是標題、內容、發布日期，因此我們要用3個變數暫存再放進list中
        title_tmp, content_tmp, tmp_date = get_info(sub_soup_main)
        # 將多個空白改成一個空白 
        title.append((' '.join(title_tmp.split())))
        content.append((' '.join(content_tmp.split())))
        out.append([int(ID[j]), tmp_date, (' '.join(title_tmp.split())), (' '.join(content_tmp.split())), link[j]])
    print('afa_news crawler finish')  

    print('afa_news Text mining start')
    df_title, df_content = news_jieba("afa", ID, title, content)
    print('afa_news Text mining finish')

    # 建立連線並定義collections名稱
    client = MongoClient('mongodb+srv://{}:{}@twfruit.i2omj.mongodb.net/myFirstDatabase?retryWrites=true&w=majority'.format(mongodb_atlas_account, mongodb_atlas_password))
    db = client.TWFruits
    
    afa_news_title_jieba = db.afa_news_title_jieba
    afa_news_content_jieba = db.afa_news_content_jieba
    afa_news = db.afa_news

    # 定義數字型態的columns並將該欄位所有rows轉換成數字型態
    news_col = ['afa_id', 'date', 'title', 'content', 'link']
    title_cols = ['afa_id','value_1', 'value_2', 'value_3', 'value_4']
    content_cols = ['afa_id', 'value_1','value_2', 'value_3','value_4', 'value_5',
                   'value_6', 'value_7',  'value_8',  'value_9',  'value_10',
                  'value_11', 'value_12', 'value_13', 'value_14', 'value_15',
                  'value_16', 'value_17', 'value_18', 'value_19', 'value_20']
    df_afa_news = pd.DataFrame(np.array(out), columns=news_col)

    df_title[title_cols] = df_title[title_cols].apply(pd.to_numeric)
    df_content[content_cols] = df_content[content_cols].apply(pd.to_numeric)
    df_afa_news['afa_id'] = df_afa_news['afa_id'].apply(pd.to_numeric)

    
    print("afa_news update to mongodb -> start")
    # 判斷id是否存在於mongodb中，若無則寫進資料庫
    for excist_id in df_afa_news['afa_id']:
        print(excist_id)
        if [x for x in afa_news.find({"afa_id":int(excist_id)})] == []:
            afa_news_update = df_afa_news.loc[df_afa_news["afa_id"]==excist_id].to_dict(orient='records')
            updated = afa_news.insert_one(afa_news_update[0]).inserted_id
            print("afa_news update id ", updated)

    for excist_id in df_title['afa_id']:
        if [x for x in afa_news_title_jieba.find({"afa_id":int(excist_id)})] == []:
            afa_news_title_jieba_update = df_title.loc[df_title["afa_id"]==excist_id].to_dict(orient='records')
            # print(afa_news_title_jieba_update[0])
            updated = afa_news_title_jieba.insert_one(afa_news_title_jieba_update[0]).inserted_id
            print("afa_news title jieba update id ", updated)


    for excist_id in df_content['afa_id']:
        if [x for x in afa_news_content_jieba.find({"afa_id":int(excist_id)})] == []:
            afa_news_content_jieba_update = df_content.loc[df_content["afa_id"]==excist_id].to_dict(orient='records')
            updated = afa_news_content_jieba.insert_one(afa_news_content_jieba_update[0]).inserted_id
            print("afa_news content jieba update id ", updated)

    print("afa_news update to mongodb -> finish")
    client.close()

    return

afa_news(2)

In [ ]:
def coa_news(start_year_tmp, start_month_tmp, end_year_tmp, end_month_tmp):
    
    print('coa_news crawler start') 
    form_data = {
    'keyword': '',
    'division_lv1': '*',
    'year': start_year_tmp,
    'month': start_month_tmp,
    'end_year': end_year_tmp,
    'end_month': end_month_tmp,
    'search_Submit': '查詢',
    'is_search': 'y'
    }

    headers = {
        'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36',
    }
    url = 'https://www.coa.gov.tw/theme_list.php?theme=news&sub_theme=agri'
    ss = requests.session()
    res = ss.post(url=url, headers=headers, data=form_data)
    soup = BeautifulSoup(res.text, 'html.parser')
    out = []
    ID = []
    date = []
    title = []
    author = []
    link = []
    content = []

    # 取發布日期
    for b in range(1, len(soup.select('td[align="center"]')), 3):
        newsDate = soup.select('td[align="center"]')[b]
        date.append(newsDate.text)

    # 取發布機關
    for c in range(2, len(soup.select('td[align="center"]')), 3):
        newsAuthor = soup.select('td[align="center"]')[c]
        author.append(newsAuthor.text)

    # 取新聞標題、網址
    for i in range(0, len(soup.select('a[class="main-c9-index"]'))):
        newsTitle = soup.select('a[class="main-c9-index"]')[i]['title']
        newsLink = 'https://www.coa.gov.tw/' + soup.select('a[class="main-c9-index"]')[i]['href']
        title.append(re.sub('[-:_【】)(「」&+\n\t\r\u3000\xa0]', ' ', newsTitle))
        link.append(newsLink)

    # 取新聞內容、文號
    for j in range(len(link)):
        page_res = ss.get(url=link[j], headers=headers)
        page_soap = BeautifulSoup(page_res.text, 'html.parser')

        # 災情報告文號為"HOT"，改取連結後面的id
        ID_tmp = page_soap.select('td[class="word-2"]')[0].text.split("：")[1]
        if ID_tmp.isdigit():
            ID.append(ID_tmp)
        else:
            ID.append(link[j].split("&id=")[1])

        for w in page_soap.select('div[class="word"]'):
            content.append(re.sub('[-:_【】)(「」&+\n\t\r\u3000\xa0]', ' ', w.text))

    for k in range(len(ID)):
        out.append([ID[k], date[k], author[k], title[k], content[k], link[k]])
#     print('out=', out)
    print('coa_news crawler finish')  

    # 定義數字型態的columns並將該欄位所有rows轉換成數字型態
    news_col = ['coa_id', 'date', 'author', 'title', 'content', 'link']
    title_cols = ['coa_id','value_1', 'value_2', 'value_3', 'value_4']
    content_cols = ['coa_id', 'value_1','value_2', 'value_3','value_4', 'value_5',
                   'value_6', 'value_7',  'value_8',  'value_9',  'value_10',
                  'value_11', 'value_12', 'value_13', 'value_14', 'value_15',
                  'value_16', 'value_17', 'value_18', 'value_19', 'value_20']

    df_coa_news = pd.DataFrame(np.array(out), columns=news_col)
    df_coa_news['coa_id'] = df_coa_news['coa_id'].apply(pd.to_numeric)

    # text mining
    print('coa_news Text mining start')
    df_title, df_content = news_jieba("coa", ID, title, content)
    print('coa_news Text mining finish')

    # 建立連線並定義collections名稱
    client = MongoClient('mongodb+srv://{}:{}@twfruit.i2omj.mongodb.net/myFirstDatabase?retryWrites=true&w=majority'.format(mongodb_atlas_account, mongodb_atlas_password))
    db = client.TWFruits
    
    coa_news_title_jieba = db.coa_news_title_jieba
    coa_news_content_jieba = db.coa_news_content_jieba
    coa_news = db.coa_news


    df_title[title_cols] = df_title[title_cols].apply(pd.to_numeric)
    df_content[content_cols] = df_content[content_cols].apply(pd.to_numeric)
    
    # Demo
    print(df_coa_news)
    print("--------------------")
    print(df_title)
    print("====================")
    print(df_content)

    # 判斷id是否存在於mongodb中，若無則insert
    print("coa_news update to mongodb -> start")

    for excist_id in df_coa_news['coa_id']:
        if [x for x in coa_news.find({"coa_id":int(excist_id)})] == []:
            coa_news_update = df_coa_news.loc[df_coa_news["coa_id"]==excist_id].to_dict(orient='records')
            updated = coa_news.insert_one(coa_news_update[0]).inserted_id
            print("coa_news update id ", updated)

    for excist_id in df_title['coa_id']:
        if [x for x in coa_news_title_jieba.find({"coa_id":int(excist_id)})] == []:
            coa_news_title_jieba_update = df_title.loc[df_title["coa_id"]==excist_id].to_dict(orient='records')
    #         print(afa_news_title_jieba_update[0])
            updated = coa_news_title_jieba.insert_one(coa_news_title_jieba_update[0]).inserted_id
            print("coa_news title jieba update id ", updated)


    for excist_id in df_content['coa_id']:
        if [x for x in coa_news_content_jieba.find({"coa_id":int(excist_id)})] == []:
            coa_news_content_jieba_update = df_content.loc[df_content["coa_id"]==excist_id].to_dict(orient='records')
            updated = coa_news_content_jieba.insert_one(coa_news_content_jieba_update[0]).inserted_id
            print("coa_news content jieba update id ", updated)

    print("coa_news update to mongodb -> finish")
    client.close()

    return

coa_news(110, 4, 110, 6)

In [ ]:
def marketing_price_soup(fruit_name, table_content_tmp):

    Header = table_content_tmp[0:10]
    Header[7] = '價格跟前一交易日比較%'
    Header[9] = '交易量跟前一交易日比較%'
    print('marketing_price crawler -> finish') 
    
    #清洗資料
    data = table_content_tmp[18:]
    output=[]
    for s_data in range(0, len(data), 10):
        output.append(data[s_data:s_data+10])    
    df3 = pd.DataFrame(output,columns=Header)
    # 按日期排序，reset_index後會多出index欄位，後面讀回的原始資料沒有index因此一併去除
    df3 = df3.sort_values(["日期"], ascending=True).reset_index().drop(['index'], axis=1)
    
    # 將交易量中的","去除並轉成數字
    df3["交易量(公斤)"] = df3["交易量(公斤)"].apply(lambda x: int(" ".join(re.sub(",", "", x).split())))
    
    # dataFrame部分欄位轉成數字型態
    col = ["上價", "中價", "下價", "平均價(元/公斤)", "交易量(公斤)"]
    df3[col] = df3[col].apply(pd.to_numeric)
    
    df3['市場'] = df3['市場'].apply(lambda x: x.rstrip(" ").replace(" ", "-"))
    df3['產品'] = df3['產品'].apply(lambda x: x.rstrip(" ").replace(" ", "-"))
#     df3.to_csv("farmproduct_{}_crawler.csv".format(fruit_name), index=False)
    
    print("marketing_price update to mongodb -> start")
    # 寫進mongodb
    client = MongoClient('mongodb+srv://{}:{}@twfruit.i2omj.mongodb.net/myFirstDatabase?retryWrites=true&w=majority'.format(mongodb_atlas_account, mongodb_atlas_password))
    db = client.TWFruits
    marketing_price_data = db.price_marketing
    
    #用日期與市場欄位判斷資料是否已存在，若無則寫入資料庫
    for excist_time, excist_marketing, excist_fruit,  in zip(df3['日期'], df3['市場'], df3['產品']):
#         print(excist_time,excist_marketing)
        if [x for x in marketing_price_data.find({"日期":excist_time, "市場":excist_marketing, "產品":excist_fruit})] == []:
            marketing_price_data_update = df3.loc[(df3["日期"]==excist_time) & (df3["市場"]==excist_marketing) & (df3["產品"]==excist_fruit)].to_dict(orient='records')
            updated = marketing_price_data.insert_one(marketing_price_data_update[0]).inserted_id
            print("marketing_price_data update id ", updated)
    print("marketing_price update to mongodb -> finish")
    
    return

In [ ]:
def marketing_price(fruit, start_date):

    fruit = "guava"
    start_date  = '109/01/01'

    print('marketing_price crawler -> start') 
    fruit_select = {"banana":"A1","pineapple":"B2","guava":"P1"}
    url = "https://amis.afa.gov.tw/fruit/FruitProdDayTransInfo.aspx"

    start_time = time.time()

    chrome_options = Options()
    chrome_options.add_argument("--headless")
    driver = webdriver.Chrome(options=chrome_options)

    # driver = Chrome("./chromedriver")
    driver.get(url)

    #### 1. 選取範圍 => 期間
    driver.find_element_by_xpath("//*[@id='ctl00_contentPlaceHolder_ucDateScope_rblDateScope_1']").click()
    time.sleep(2)

    #### 2. 選取日期
    # 執行js語法來解除只能read的input格子
    driver.execute_script("$('input[id=ctl00_contentPlaceHolder_txtSTransDate]').removeAttr('readonly')")

    # 清空既有input並放入keys
    driver.find_element_by_id('ctl00_contentPlaceHolder_txtSTransDate').clear() 
    driver.find_element_by_id('ctl00_contentPlaceHolder_txtSTransDate').send_keys(start_date)
    # time.sleep(2)

    #### 3. 選取市場(目前code僅能選全部市場)
    driver.find_element_by_xpath("//*[@id='ctl00_contentPlaceHolder_txtMarket']").click() 
    #time.sleep(3)

    # 點選後轉移到跳出視窗，選取全部市場
    iframe = driver.find_elements_by_tag_name("iframe")[0]
    driver.switch_to.frame(iframe)
    radio_target = driver.find_element_by_xpath("//*[@id='radlMarketRange_0']")
    radio_target.click()

    #### 4. 選取水果種類 
    driver.find_element_by_xpath("//*[@id='ctl00_contentPlaceHolder_txtProduct']").click()
    #time.sleep(3)

    # 點選後轉移到跳出視窗
    iframe = driver.find_elements_by_tag_name("iframe")[0]
    driver.switch_to.frame(iframe)

    # 抓取下拉選單元件，直接以值來選擇
    select = Select(driver.find_element_by_name('lstProduct'))
    select.select_by_value(fruit_select[fruit])

    # 選取完成後，關閉視窗
    driver.find_element_by_xpath("//*[@id='btnConfirm']").click()

    #### 5. 點選查詢button
    driver.find_element_by_xpath("//*[@id='ctl00_contentPlaceHolder_btnQuery']").click()

    # 有時候網頁跑很久，sleep太少會拿到錯誤的page_source
    time.sleep(5)
    #     html = driver.execute_script("return document.getElementsByTagName('html')[0].outerHTML")

    soup = BeautifulSoup(driver.page_source,'html.parser')

    # time.sleep(60)

    table = soup.select("table[border='1']")
    table_content = list(filter(None,table[0].text.split('\n')))

    marketing_price_soup(fruit, table_content)
    
marketing_price("guava", '109/01/01')   

In [ ]:
weather_all(StartYear):
    col = ['ObsTime', 'StnPres', 'SeaPres', 'StnPresMax', 'StnPresMaxTime','StnPresMin',
           'StnPresMinTime', 'Temperature', 'TMax', 'TMaxTime', 'TMin', 'TMinTime',
           'Td dew point', 'RH', 'RHMin', 'RHMinTime', 'WS', 'WD', 'WSGust','WDGust',
           'WGustTime', 'Precp', 'PrecpHour', 'PrecpMax10', 'PrecpMax10Time', 'PrecpMax60',
           'PrecpMax60Time', 'SunShine', 'SunShineRate', 'GloblRad', 'VisbMean',
           'EvapA', 'UVIMax', 'UVIMaxTime', 'CloudAmount']

    resource_path = r'./WeatherDatas'
    if not os.path.exists(resource_path):
        os.mkdir(resource_path)

    today = datetime.datetime.now()
    NowYear = today.year
    NowMonth = today.month
    NowDay = today.day

    client = MongoClient('mongodb+srv://{}:{}@twfruit.i2omj.mongodb.net/myFirstDatabase?retryWrites=true&w=majority'.format(mongodb_atlas_account, mongodb_atlas_password))
    db = client.TWFruits
    weather_data = db.climate_weather

    Error = []
    with open('./現存測站3.csv',encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        column = [row for row in reader]

        for i in range(0,len(column)):
            StationNumber = column[i]['站號']
            StationName = column[i]['站名']
            StationLocation = column[i]['城市']
            Remark = column[i]['備註']
            # StartYear = column[i]['資料起始日期'].split('/')[0]
            # StartMonth = column[i]['資料起始日期'].split('/')[1]

            if Remark == "本站只有雷達觀測資料。":
                print('{} only have radar observation data'.format(StationName))
            else :
                Yearlist = []
                Monthlist = []
                for year in range(StartYear, NowYear + 1):
                    if year == NowYear:
                        for j in range(1, NowMonth + 1):
                            if len(str(j)) == 1:
                                month = "0" + str(j)
                            else:
                                month = j
                            Yearlist.append(year)
                            Monthlist.append(month)
                    else:
                        for j in range(1, 13):
                            if len(str(j)) == 1:
                                month = "0" + str(j)
                            else:
                                month = j
                            Yearlist.append(year)
                            Monthlist.append(month)
                print('======{} is START!======'.format(StationName))
                for k in range(0, len(Yearlist)):
                    filepath = r'{}/{}/{}/{}.csv'.format(resource_path, StationLocation, StationName, StationName + "{}{}".format(Yearlist[k], Monthlist[k]))
                    ## 看是否存在
                    if os.path.exists(filepath):
                        print('{} is exist!!'.format(StationName + "{}{}".format(Yearlist[k], Monthlist[k])))
                    else:
                        print("==========================")
                        print(Yearlist[k], Monthlist[k])
                        url = "http://e-service.cwb.gov.tw/HistoryDataQuery/MonthDataController.do?command=viewMain&station={}&stname=%25E9%259E%258D%25E9%2583%25A8&datepicker={}-{}".format(
                            StationNumber, Yearlist[k], Monthlist[k])
                        r = requests.get(url)
                        r.encoding = "utf-8"
                        soup = BeautifulSoup(r.text, "html.parser")
                        tag_table = soup.find(id="MyTable")
                        rows = tag_table.findAll("tr")
                        rowList_org = [[cell.get_text().replace("\n", "").replace("\r", "") for cell in row.findAll(["td"])] for row in rows]

                        # 去除前三行空值
                        list(map(lambda x: rowList_org.pop(0), range(3)))
                        rowList = []
                        # 將所有欄位去除空白->“\xa0”
                        for t in range(len(rowList_org)): # 天數
                            rowList_t = list(map(lambda x: rowList_org[t][x].split('\xa0')[0], range(len(rowList_org[0])))) # 該天所有欄位
                            rowList.append(rowList_t)

                        resource_path2 = r'{}/{}'.format(resource_path, StationLocation)
                        if not os.path.exists(resource_path2):
                            os.mkdir(resource_path2)
                        resource_path3 = r'{}/{}'.format(resource_path2, StationName)
                        if not os.path.exists(resource_path3):
                            os.mkdir(resource_path3)

                        try:
                            df = pd.DataFrame(rowList,
                                              columns=col)

                            df.to_csv(r'%s/%s.csv' % (resource_path3, StationName + "{}{}".format(Yearlist[k], Monthlist[k])), index=False)
                            print('OK: {}/{}'.format(Yearlist[k], Monthlist[k]))
                            data = rowList
                            data.insert(0, col)
                            weather_data_update = {"year":Yearlist[k], "month":int(Monthlist[k]) ,"station": StationName, "data":data}

                            # 依年分、測站名稱、觀測要素尋找是否存在mongodb，若無則新增進mongodb
                            if [x for x in weather_data.find({"year":Yearlist[k], "month":int(Monthlist[k]), "station": StationName})] == []:
                                updated = weather_data.insert(weather_data_update)
                                print("weather_data update id ", updated)
                            else:
                                print("data exist in mongodb -> no need to update")
                            print("weather_data update to mongodb -> finish")
                            time.sleep(2)
                        except ValueError as e:
                            print("ValueError:", url)
                            Error.append(url)
                        except IndexError as e:
                            print("IndexError:", url)
                            Error.append(url)
                        print("==========================")

                print('======{} is over======'.format(StationName))
                time.sleep(2)
        with open('{}/Error_{}{}.txt'.format(resource_path, NowMonth, NowDay), "w") as f:
            for error in Error:
                f.write('{}\n'.format(error))
    return

weather_all(2010)

In [ ]:
def wether_today():
    col = ['ObsTime', 'StnPres', 'SeaPres', 'StnPresMax', 'StnPresMaxTime','StnPresMin',
           'StnPresMinTime', 'Temperature', 'TMax', 'TMaxTime', 'TMin', 'TMinTime',
           'Td dew point', 'RH', 'RHMin', 'RHMinTime', 'WS', 'WD', 'WSGust','WDGust',
           'WGustTime', 'Precp', 'PrecpHour', 'PrecpMax10', 'PrecpMax10Time', 'PrecpMax60',
           'PrecpMax60Time', 'SunShine', 'SunShineRate', 'GloblRad', 'VisbMean',
           'EvapA', 'UVIMax', 'UVIMaxTime', 'CloudAmount']

    resource_path = r'./WeatherDatas'
    if not os.path.exists(resource_path):
        os.mkdir(resource_path)

    today = datetime.datetime.now()
    NowYear = today.year
    NowMonth = today.month
    NowDay = today.day

    client = MongoClient('mongodb+srv://{}:{}@twfruit.i2omj.mongodb.net/myFirstDatabase?retryWrites=true&w=majority'.format(mongodb_atlas_account, mongodb_atlas_password))
    db = client.TWFruits
    weather_data = db.climate_weather

    if len(str(NowMonth)) == 1:
        Month = "0" + str(NowMonth)
    else:
        Month = NowMonth

    Error = []
    with open('./現存測站.csv',encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        column = [row for row in reader]

    for i in range(0,len(column)):
        StationNumber = column[i]['站號']
        StationName = column[i]['站名']
        StationLocation = column[i]['城市']
        Remark = column[i]['備註']

        if Remark == "本站只有雷達觀測資料。":
            print('{} only have radar observation data'.format(StationName))

        else:
            print("==========================")
            print('Update {}'.format(StationName + "{}{}".format(NowYear, Month)))
            url = "http://e-service.cwb.gov.tw/HistoryDataQuery/MonthDataController.do?command=viewMain&station={}&stname=%25E9%259E%258D%25E9%2583%25A8&datepicker={}-{}".format(StationNumber, NowYear, Month)
            r = requests.get(url)
            r.encoding = "utf-8"
            soup = BeautifulSoup(r.text, "html.parser")
            tag_table = soup.find(id="MyTable")
            rows = tag_table.findAll("tr")
            rowList_org = [[cell.get_text().replace("\n", "").replace("\r", "") for cell in row.findAll(["td"])] for row in rows]

            # 去除前三行空值
            list(map(lambda x: rowList_org.pop(0), range(3)))
            rowList = []
            # 將所有欄位去除空白->“\xa0”
            for t in range(len(rowList_org)): # 天數
                rowList_t = list(map(lambda x: rowList_org[t][x].split('\xa0')[0], range(len(rowList_org[0])))) # 該天所有欄位
                rowList.append(rowList_t)

            resource_path2 = r'{}/{}'.format(resource_path, StationLocation)
            if not os.path.exists(resource_path2):
                os.mkdir(resource_path2)
            resource_path3 = r'{}/{}'.format(resource_path2, StationName)
            if not os.path.exists(resource_path3):
                os.mkdir(resource_path3)

            try:
                df = pd.DataFrame(rowList,
                                  columns=col)

                df.to_csv(
                    r'%s/%s.csv' % (resource_path3, StationName + "{}{}".format(NowYear, Month)),
                    index=False)
                print('OK: {}/{}'.format(NowYear, Month))
                data = rowList.insert(0, col)
                weather_data_update = {"year":NowYear, "month":int(Month) ,"station": StationName, "data":data}
                # 依年分、測站名稱、觀測要素尋找是否存在mongodb，若無則新增進mongodb
                if [x for x in weather_data.find({"year":NowYear, "month":int(Month), "station": StationName})] == []:
                    updated = weather_data.insert(weather_data_update)
                    print("weather_data update id ", updated)
                else:
                    print("data exist -> no need to update")
                print("weather_data update to mongodb -> finish")
                time.sleep(1)

            except ValueError as e:
                print("ValueError:", url)
                Error.append(url)
            except IndexError as e:
                print("IndexError:", url)
                Error.append(url)

            print('Update {} is done'.format(StationName + "{}{}".format(NowYear, Month)))
            print("==========================")
            time.sleep(2)
    client.close() 

    with open('{}/Error_DailyUpdate_{}{}.txt'.format(resource_path, NowMonth, NowDay), "w") as f:
        for error in Error:
            f.write('{}\n'.format(error))

wether_today()

In [ ]:
def origin_price(fruit, StartYear_tmp, StartMonth_tmp, EndYear_tmp, EndMonth_tmp):

    fruit = "番石榴"
    StartYear_tmp = 2010
    StartMonth_tmp = 1
    EndYear_tmp = 2020
    EndMonth_tmp = 12

    fruit_type = {"番石榴":"WR1_2_PRMG_02_6", "香蕉":"WR1_2_PRMG_02_23", "鳳梨":"WR1_2_PRMG_02_25"}

    start_time = time.time()

    print('origin_price crawler -> start') 
    url = "https://apis.afa.gov.tw/pagepub/AppContentPage.aspx?itemNo=PRI075"
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    driver = webdriver.Chrome(options=chrome_options)
    # driver = Chrome('./chromedriver')
    driver.get(url)

    driver.find_element_by_xpath("//*[@id='PRI105']").click()
    time.sleep(5)  
    ###選旬平均
    driver.find_element_by_xpath("//*[@id='WR1_2_Q_AvgPriceType_C1_2']").click()
    time.sleep(5)
    ###開始年
    select = Select(driver.find_element_by_name('WR1_2$Q_PRSR_Year1$C1'))
    select.select_by_visible_text(u"{}".format(StartYear_tmp))
    ###開始月
    time.sleep(1)
    select = Select(driver.find_element_by_name('WR1_2$Q_PRSR_Month1$C1'))
    select.select_by_visible_text(u"{}".format(StartMonth_tmp))
    ###結束年
    time.sleep(1)
    select = Select(driver.find_element_by_name('WR1_2$Q_PRSR_Year2$C1'))
    select.select_by_visible_text(u"{}".format(EndYear_tmp))
    ###結束月
    time.sleep(1)
    select = Select(driver.find_element_by_name('WR1_2$Q_PRSR_Month2$C1'))
    select.select_by_visible_text(u"{}".format(EndMonth_tmp))
    ###選種類
    time.sleep(1)
    driver.find_element_by_xpath("//input[@id='{}']".format(fruit_type[fruit])).click()

    ###查詢
    time.sleep(1)
    driver.find_element_by_xpath("//*[@class='CSS_ABS_NormalLink']").click()
    print("coast time=", time.time()-start_time)

    time.sleep(5)
    window_1 = driver.current_window_handle
    windows = driver.window_handles
    for current_window in windows:
        if current_window != window_1:
            driver.switch_to.window(current_window)

    time.sleep(2)      
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    try:
        table = soup.select("table[border='1']")
    except:
        print('production price Error')

    rows = table[0].findAll("tr")
    rowList = [[cell.get_text().replace("\n", "") for cell in row.findAll(["td"])] for row in rows]
    Header = rowList[0]
    Content = rowList[1:len(rowList)]
    df = pd.DataFrame(Content, columns=Header)
    print('origin_price crawler -> finish')

    # 初步資料清潔
    df_transposed = df.set_index('地點').T
    df_transposed = df_transposed.reset_index()
    df_transposed = df_transposed.rename(columns={"index": "時間"})

    col = []
    for i in range(1, len(df_transposed.columns)):
        col.append(df_transposed.columns[i])

    df_transposed[df_transposed[col] == '-'] = 0
    df_transposed[col] = df_transposed[col].apply(pd.to_numeric)
    # 清掉->(元/公斤)
    df_transposed["時間"] = df_transposed["時間"].apply(lambda x: x.split("(")[0])

    #     df_transposed.to_csv("{}_origin_price.csv".format(fruit), index=False)

    # 增加水果種類再寫進資料庫以區分水果類別
    df_transposed['水果種類'] = fruit
    cols = df_transposed.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    df_transposed = df_transposed[cols]

    print(df_transposed)

    # print("origin_price update to mongodb -> start")
    # client = MongoClient('mongodb+srv://{}:{}@twfruit.i2omj.mongodb.net/myFirstDatabase?retryWrites=true&w=majority'.format(mongodb_atlas_account, mongodb_atlas_password))
    # db = client.TWFruits
    # origin_price_data = db.price_origin

    # for excist_time, excist_fruit, in zip(df_transposed['時間'], df_transposed["水果種類"]):
    #     if [x for x in origin_price_data.find({"時間":excist_time, "水果種類":excist_fruit})] == []:
    #         origin_price_data_update = df_transposed.loc[(df_transposed["時間"]==excist_time) & (df_transposed["水果種類"]==excist_fruit)].to_dict(orient='records')
    #         updated = origin_price_data.insert_one(origin_price_data_update[0]).inserted_id
    #         print("origin_price_data update id ", updated)
    #     else:
    #         print("excist")
    
origin_price("番石榴", 2020, 1, 2020, 12)

In [ ]:
# code是正常的，但不知道為啥有一次執行jupyter直接掛掉@@
def agriculture_survey():
    print("agriculture_survey crawler start")
    url = "https://data.coa.gov.tw/Service/OpenData/FromM/TownCropData.aspx"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36'
            }

    res = requests.get(url, headers=headers)
    data = json.loads(res.text)
    df = pd.json_normalize(data)
    print("agriculture_survey crawler finish")

    print("agriculture_survey update to mongodb -> start")
#     client = MongoClient()
#     db = client.test
    client = MongoClient('mongodb+srv://{}:{}@twfruit.i2omj.mongodb.net/myFirstDatabase?retryWrites=true&w=majority'.format(mongodb_atlas_account, mongodb_atlas_password))
    db = client.TWFruits
    
    agriculture_survey_data = db.produce_agricultural_survey

    agriculture_survey_update = df.to_dict(orient='records')
    agriculture_survey_data.insert_many(agriculture_survey_update)
    print("agriculture_survey update to mongodb -> finish")
    
agriculture_survey()  

In [ ]:
def fruit_season():
    print("Fruit_season crawler start")
    url = "https://data.coa.gov.tw/Service/OpenData/DataFileService.aspx?UnitId=061&$top=6000&$skip=0" # "https://data.coa.gov.tw/Service/OpenData/DataFileService.aspx?UnitId=113" #"https://data.coa.gov.tw/Service/OpenData/FromM/TownCropData.aspx"
    headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36'
                }

    res = requests.get(url, headers=headers)
    data = json.loads(res.text)
    print("Fruit_season crawler finish")
    
    df = pd.json_normalize(data)
    df_fruit = df.loc[df["type"] == '水果']
    df_fruit["month"] = df_fruit["month"].apply(pd.to_numeric)
#     df_fruit["month"].
    
    print("Fruit_season update to mongodb -> start")
    client = MongoClient('mongodb+srv://{}:{}@twfruit.i2omj.mongodb.net/myFirstDatabase?retryWrites=true&w=majority'.format(mongodb_atlas_account, mongodb_atlas_password))
    db = client.TWFruits
    fruit_season_data = db.produce_fruit_season

    df_fruit_data_update = df_fruit.to_dict(orient='records')
    fruit_season_data.insert_many(df_fruit_data_update)
    print("fruit_season update to mongodb -> finish")
    
fruit_season()

In [ ]:
def agriculture_weather(start_year):
    start_year = '2021-01-01'
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36',
               }
    arg_weather_url = "https://agr.cwb.gov.tw/NAGR/history/station_day"

    area_select = ["南部", "中部"]

    south_station = ["東港工作站(12Q97)", "恆春工作站(12Q98)", "義竹分場(72M36)", "嘉義分場(72M70)", 
                     "臺南農改(72N10)", "七股研究中心(72N24)", "高雄農改(72Q01)", "旗南農改(72V14)", 
                     "臺南蘭花園區(A2N29)", "畜試所(B2N89)", "恆春畜試(B2Q81)", "林試六龜中心(E2P98)", 
                     "林試扇平站(E2P99)", "嘉義農試(G2L02)", "溪口農場(G2M35)", "鳳山農試(G2P82)"]

    center_station = ["口湖工作站(12J99)", "臺中農改(72G60)", "埔里分場(72HA0)", "雲林分場(72K22)",
                      "魚池茶改(82H32)", "凍頂茶改(82H84)", "臺西水試所(A2K36)",
                      "臺大雲林校區(A2K63)", "蓮華池(E2H36)", "林試畢祿溪站(E2HA2)", "四湖植物園(E2K60)",
                      "農業試驗所(G2F82)", "種苗繁殖(K2F75)", "臺大溪頭(U2H48)", "臺大和社(U2HA3)", 
                      "臺大內茅埔(U2HA4)", "臺大竹山(U2HA5)"]

    error = []
    # 建立資料夾
    Agricultural_resource_path = r'./Agricultural_Weather_Datas'
    if not os.path.exists(Agricultural_resource_path):
        os.mkdir(Agricultural_resource_path)
    start_time = time.time()

    # chrome_options = Options()
    # chrome_options.add_argument("--headless")
    # driver = webdriver.Chrome(options=chrome_options)
    driver = Chrome("./chromedriver")
    driver.get(arg_weather_url)
    action = ActionChains (driver)

    # mongodb setting
    client = MongoClient('mongodb+srv://{}:{}@twfruit.i2omj.mongodb.net/myFirstDatabase?retryWrites=true&w=majority'.format(mongodb_atlas_account, mongodb_atlas_password))
    db = client.TWFruits
    agriculture_weather_data = db.climate_agriculture_weather

    # 選擇測站類別
    print("start")
    select = Select(driver.find_element_by_name('station_level'))
    select.select_by_visible_text('農業站')

    # 選擇地區
    for area in range(0, len(area_select)):

        # 爬第二次需要先刷新介面
        if area > 0:
            driver.get(arg_weather_url)
            action = ActionChains(driver)
            time.sleep(3)

        select = Select(driver.find_element_by_name('area_name'))
        select.select_by_visible_text(area_select[area])
        print("area=", area_select[area])

        # 依地區選擇測站
        if area == 0:
            station_select = south_station
        if area == 1:
            station_select = center_station

        for station_number in range(len(station_select)):
            time.sleep(1)

            # 爬第二次需要刷新介面
            if (station_number > 0) & (area > 0):
                driver.get(arg_weather_url)
                action = ActionChains(driver)
                time.sleep(3)

            select = Select(driver.find_element_by_name('station_name'))
            select.select_by_visible_text(station_select[station_number])
            print("station=", station_select[station_number])
            time.sleep(1)

            # 選擇日期
            driver.execute_script("$('input[id=start_time]').removeAttr('readonly')")
            driver.find_element_by_id('start_time').clear() 
            driver.find_element_by_id('start_time').send_keys(start_year)
            time.sleep(1)

            # 對所有觀測要素逐一爬蟲
            for item_number in range(1, 34): # max item -> 33
                print("item_number=",item_number)
                # ActionChains重新抓取目前的網頁
                action = ActionChains(driver)
                try:
                    item_click = driver.find_element_by_xpath('//*[@id="item_multiple"]/option[{}]'.format(item_number))
                except:
                    # 要素若無觀測資料則重新選擇
                    print("Error")
                    driver.get(arg_weather_url)
                    action = ActionChains (driver)
                    select = Select(driver.find_element_by_name('station_level'))
                    select.select_by_visible_text('農業站')
                    time.sleep(1)
                    select = Select(driver.find_element_by_name('area_name'))
                    select.select_by_visible_text(area_select[area])
                    time.sleep(1)
                    select = Select(driver.find_element_by_name('station_name'))
                    select.select_by_visible_text(station_select[station_number])
                    time.sleep(1)
                    driver.execute_script("$('input[id=start_time]').removeAttr('readonly')")
                    driver.find_element_by_id('start_time').clear() 
                    driver.find_element_by_id('start_time').send_keys(start_year)
                    continue

                # 按下預覽   
                action.click(item_click).release(item_click).perform()
                time.sleep(1)
                driver.find_element_by_xpath('//*[@id="create_table"]').click()
                time.sleep(1)

                #若 station 取值正常則脫離迴圈繼續，取值異常可能是網站還沒跑完，暫停1秒再次取值，迴圈重複3次
                for t in range(3):
                    # 取得html
                    html = driver.execute_script("return document.getElementsByTagName('html')[0].outerHTML")
                    soup = BeautifulSoup(driver.page_source,'html.parser')

                    # all_tale -> 包含表格標題及註解等雜項，用來取得站名、年分、觀測要素名稱
                    # table -> 僅有各年份表格內容，用來彙整觀測資訊並轉換成df輸出
                    all_table = soup.select("div[id='station_day_table']")
                    table = soup.select("table[style='width:100%']")
                    # 測站名稱
                    try:
                        station = all_table[0].select("tr")[0].text.split("測站：")[1]
                        break
                    except:
                        print('get station {} time fail'.format(t))
                        time.sleep(1)
                        continue

                # 取得並整理 dataframe columns
                # 取得"年分"
                year_list = []
                for i in range(len(table)): # 3
                    year_list.append(all_table[0].select("tr")[i*44+1].text.split("年份：")[1])
                print(year_list)  

                col_list = list(map(lambda x: table[0].select("tr")[0].select("th")[x].text, range(len(table[0].select("tr")[0]))))
                # 去掉最後一個columns(平均)
                list_pop = col_list.pop(-1)
                col = list(map(lambda x: col_list[x]+"月", range(len(col_list))))
                # 取得觀測要素名稱，並將有"/"的剔除以免被當成路徑
                item_tmp = all_table[0].select("tr")[1].text.split("：")[1].split("，")[0]
                item = re.sub("/", "", item_tmp)

                #建立各站資料夾
                station_path = r'{}/{}'.format(Agricultural_resource_path, station)
                if not os.path.exists(station_path):
                    os.mkdir(station_path)
                data_all = []

                print("data organize")
                # 照各年分依序爬蟲
                for i in range(len(table)): # 年分
                    data = []
                    col[0] = "{}年 ".format(year_list[i]) + "日/月"
                    for j in range(len(table[i].select("tr"))): # 33
                        day = table[i].select("tr")[j].select("th")[0].text
                        row = list(map(lambda x: table[i].select("tr")[j].select("td")[x].text, range(len(table[i].select("tr")[j].select("td")))))

                        # 有抓到12個月份的資料(包含"/"及空值)
                        if len(row) == 12:
                            row.insert(0, day)
                            data.append(row)

                    print(col)
                    print(data)
                    data_all.append(data)
                    print("updating {} {} {} data".format(station, year_list[i], item))
                    # print("station_path=", station_path)
                    # print(df.head())
                    df = pd.DataFrame(data_all[i], columns=col)
                    print(df.head(3))
                    # 看每月平均是否有值，當check==True代表該年分有資料紀錄，需要被更新
                    check = list(map(lambda x: df[col[x]][31] not in ['/',' ',''], range(1, len(col))))
#                     if True in check:
#                         print("output to csv")
#                         df.to_csv('{}/{}{}.csv'.format(station_path ,year_list[i], item),index=False)

#                         # mongodb
#                         print("agriculture_weather update to mongodb -> start")
#                         # 將氣候資料加入columns，並用字典將 年分、測站名稱、觀測要素、資料打包
#                         data.insert(0, col)
#                         agriculture_weather_data_update = {"year":int(year_list[i]), "station": station, "item": item, "data":data}
#                         # 依年分、測站名稱、觀測要素尋找是否存在mongodb，若無則新增進mongodb
#                         if [x for x in agriculture_weather_data.find({"year": int(year_list[i]), "station": station, "item": item})] == []:
#                             updated = agriculture_weather_data.insert(agriculture_weather_data_update)
#                             print("agriculture_weather update id ", updated)
#                         else:
#                             print("data exist -> no need to update")
#                         print("agriculture_weather update to mongodb -> finish")

#                     else:
#                         print("{} No data".format(year_list[i]))
#                     print("done")
    driver.close()
    client.close() 
    return
agriculture_weather('2010-01-01')

In [ ]:
# main
#select crawler

# 水果產季
fruit_season()

# 農情調查
agriculture_survey()

# 颱風警報
get_typhoon_alert()

# 年度生產愾況
produce_year() 

# 市場價格 (種類, 開始日期(民國年/月/日))
marketing_price("香蕉", '110/05/01')

# 新聞 (開始年, 開始月, 結束年, 結束月)
coa_news(110, 1, 110, 6)

# 新聞 (總爬取頁數)
afa_news(2) 

# 氣象資料(開始年分)
wether(2021)

# 農業測站氣象資料
agriculture_weather('2010-01-01')

# 產地價格
origin_price(2020, 1, 2020, 12)


In [ ]:
# connect to mongodb
import pymongo

client = pymongo.MongoClient("mongodb+srv://<username>:<password>@<db名稱>.i2omj.mongodb.net/myFirstDatabase?retryWrites=true&w=majority")
db = client.db_name
test = db.test #測試用的collection

In [ ]:
client = MongoClient('mongodb+srv://{}:{}@twfruit.i2omj.mongodb.net/myFirstDatabase?retryWrites=true&w=majority'.format(mongodb_atlas_account, mongodb_atlas_password))
db = client.TWFruits
afa_news = db.afa_news

# x = afa_news.find()
data = [x for x in afa_news.find()]
data

In [ ]:
data[0].keys()

In [ ]:
for n in data:
    print(n['afa_id'])